In [1]:
from datetime import datetime
print(str(datetime.now()))
import numpy as np
# os.environ["TF_CPP_MIN_LOG_LEVEL"] ="3"
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import math

import os
import glob
import matplotlib.pyplot as plt


2022-09-28 20:32:42.132855


2022-09-28 20:32:42.366680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 20:32:42.366712: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# !pip install matplotlib

In [3]:
gpu_available = tf.test.is_gpu_available()
gpu_available
req_cols = ['ITEM_ID', 'USER_ID', 'CABIN_TYPE', 'USER_RESIDENCE']

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-09-28 20:32:44.522006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-28 20:32:44.522031: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-28 20:32:44.522051: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bioss-System-Product-Name): /proc/driver/nvidia/version does not exist


In [4]:
data_set_df_updated = pd.read_csv("dataset/interaction_demo.csv")
test_df = pd.read_csv("dataset/interaction_test_demo.csv")
data_set_df_updated.loc[data_set_df_updated.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
test_df.loc[test_df.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
train_df = pd.concat([data_set_df_updated, test_df], ignore_index=True)
train_df.sort_values("TIMESTAMP", ascending= False, inplace= True)

In [5]:
train_df_item_count = train_df.groupby(["ITEM_ID"]).size().reset_index(name='counts')

In [6]:
train_df_item_count["probability"]= train_df_item_count["counts"] / train_df_item_count["counts"].sum()

In [7]:
train_df = train_df.merge(train_df_item_count[["ITEM_ID","probability"]], how='left', on='ITEM_ID',)


In [8]:
item_df = train_df[["ITEM_ID"]].drop_duplicates("ITEM_ID")
item_ds = tf.data.Dataset.from_tensor_slices(item_df.to_dict("list")).batch(32)
item_df.shape

(350, 1)

In [9]:
req_cols = req_cols+["probability"]

In [10]:
train_ds =  tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list")).batch(256)

In [11]:
USER_ID_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_ID"]))))

CABIN_TYPE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["CABIN_TYPE"]))))

USER_RESIDENCE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_RESIDENCE"]))))

In [12]:
item_unique =  np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["ITEM_ID"]))))

In [13]:
# def rankL(np_rank):
#     r = int(np_rank[-1])
#     _l = 0
#     for k in range(1, r+1):
#         _l += 1./k
#     return np.float32(_l)


# """
# labels are assumed to be 1 hot encoded
# """
# def warp_loss(labels, logits):
#     # for easy broadcasting
#     labels, logits = tf.transpose(labels, [1, 0]), tf.transpose(logits, [1, 0])
#     f_y = tf.reduce_sum(logits*labels, axis=0)
#     rank = tf.reduce_sum(tf.maximum(tf.sign(1+logits-f_y), 0), axis=0)
#     diff = tf.reduce_sum(tf.maximum(1+logits-f_y, 0), axis=0)
#     with tf.control_dependencies([tf.assert_greater(rank, tf.zeros_like(rank))]):
#         return tf.py_func(rankL, [rank], tf.float32) * diff/rank

In [14]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        emb_dim = 8    
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_ID_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_ID_unique) + 1, 16),
        ])
            
        self.cabin_type_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary= CABIN_TYPE_unique, mask_token=None),  
            tf.keras.layers.Embedding(len(CABIN_TYPE_unique) + 1, emb_dim),
        ])

        self.user_residence_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_RESIDENCE_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_RESIDENCE_unique) + 1, emb_dim),
        ])
        

    def call(self, user_interation_data):
        return tf.concat([                          
            self.user_id_embedding(user_interation_data["USER_ID"]), 
            self.cabin_type_embedding(user_interation_data["CABIN_TYPE"]), 
            self.user_residence_embedding(user_interation_data["USER_RESIDENCE"]),
        ], axis=1)





In [15]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        

        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=item_unique, mask_token=None),
            tf.keras.layers.Embedding(len(item_unique) + 1, 32),
        ])


    def call(self, user_interation_data):

        return tf.concat([
            self.item_embedding(user_interation_data["ITEM_ID"])
            
            ], axis=1)

In [16]:
class TRFSRetrievalModel(tfrs.models.Model):

    def __init__(self, UserModel,ItemModel, item_ds ):
        super().__init__()



        self.query_model = tf.keras.Sequential([#,UserModel()
          UserModel(),
#           tf.keras.layers.Dense(32 , kernel_initializer= tf.keras.initializers.RandomNormal(seed=99)),  
#           tf.keras.layers.Dropout(0.2),
        ])
        

        self.candidate_model = tf.keras.Sequential([
          ItemModel(),
#           tf.keras.layers.Dense(32, kernel_initializer= tf.keras.initializers.RandomNormal(seed=1)),
#           tf.keras.layers.Dropout(0.2),
        ]) 
        
        


#         metrics = [
#           tf.keras.metrics.TopKCategoricalAccuracy(
#               k=x, name=f"factorized_top_k/top_{x}_categorical_accuracy")
#           for x in [3,5,10,15, 25]
#         ]  
        
        self.task = tfrs.tasks.Retrieval(
#             loss=warp_loss,
            num_hard_negatives=100,
            metrics=tfrs.metrics.FactorizedTopK(
            item_ds.map(self.candidate_model),
                ks= (3, 5, 10,15, 25)),
                          

        )
        
#         self.task = tfrs.tasks.Retrieval(
#             metrics=tfrs.metrics.FactorizedTopK(
#                 candidates=item_ds.map(self.candidate_model),
#                 metrics = metrics,
#                 k = 100
#             ),
#             # temperature = 0.5,
#             num_hard_negatives = 5
#         )

    def compute_loss(self, features, training= True):

        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)
        candidate_sampling_probability = features.pop("probability")
        return self.task(query_embeddings, 
        item_embeddings, 
        compute_metrics=True,
        candidate_sampling_probability = candidate_sampling_probability
        )

    def call(self, test):
        features= test.copy()
        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return query_embeddings, item_embeddings


In [17]:
test_max_index = math.floor(train_df.shape[0]*0.1)
train_split_len = train_df.shape[0] - test_max_index
data_set_tf = tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list"))
test = data_set_tf.take(test_max_index)
train = data_set_tf.skip(test_max_index).take(train_split_len)
shuffled = train.shuffle(train_split_len, seed=42, reshuffle_each_iteration=True)
cached_train = shuffled.batch(512).prefetch(4096)#train.shuffle(100_000).batch(2048)
cached_test = test.batch(512).cache()


In [18]:
train_df.shape[0], test_max_index

(34853, 3485)

In [19]:
def delete_all_check_points(fpath= 'new_amazon_check_points/*'):
    files = glob.glob(fpath)
    for f in files:
        os.remove(f)
        

In [20]:
def get_call_back_fun(K):
    delete_all_check_points()
    model_path = f"new_amazon_check_points/best_check_point_{K}k"
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        save_weights_only=True,
        monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
        mode='max',
        save_best_only=True)

    early_stoping = tf.keras.callbacks.EarlyStopping(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
                                                     mode='min',
                                                     patience=5)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy', 
                                                     factor=0.6,
                                                     #mode='min',
                                                     patience=9, 
                                                     min_lr=1e-6
    )
    return model_path, model_checkpoint_callback, early_stoping, reduce_lr

In [21]:
model_path, model_checkpoint_callback, early_stoping, reduce_lr = get_call_back_fun(25)

In [22]:
model = TRFSRetrievalModel(UserModel, ItemModel, item_ds)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01)) 

In [23]:
data = model.fit(cached_train,
          validation_data=cached_test,
          epochs=200,
          verbose=1, 
          workers=3,
          use_multiprocessing=True,
          callbacks=[model_checkpoint_callback, 
                     reduce_lr]

          )

Epoch 1/200
62/62 [==============================] - 6s 51ms/step - factorized_top_k/top_3_categorical_accuracy: 0.0409 - factorized_top_k/top_5_categorical_accuracy: 0.0767 - factorized_top_k/top_10_categorical_accuracy: 0.1273 - factorized_top_k/top_15_categorical_accuracy: 0.1778 - factorized_top_k/top_25_categorical_accuracy: 0.2770 - loss: 2882.3604 - regularization_loss: 0.0000e+00 - total_loss: 2882.3604 - val_factorized_top_k/top_3_categorical_accuracy: 0.1486 - val_factorized_top_k/top_5_categorical_accuracy: 0.2207 - val_factorized_top_k/top_10_categorical_accuracy: 0.3131 - val_factorized_top_k/top_15_categorical_accuracy: 0.3917 - val_factorized_top_k/top_25_categorical_accuracy: 0.5197 - val_loss: 1942.3702 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1942.3702 - lr: 0.0100
Epoch 2/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.1791 - factorized_top_k/top_5_categorical_accuracy: 0.2450 - factorized_top_

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8494 - factorized_top_k/top_5_categorical_accuracy: 0.9079 - factorized_top_k/top_10_categorical_accuracy: 0.9433 - factorized_top_k/top_15_categorical_accuracy: 0.9573 - factorized_top_k/top_25_categorical_accuracy: 0.9707 - loss: 1129.1514 - regularization_loss: 0.0000e+00 - total_loss: 1129.1514 - val_factorized_top_k/top_3_categorical_accuracy: 0.2290 - val_factorized_top_k/top_5_categorical_accuracy: 0.2901 - val_factorized_top_k/top_10_categorical_accuracy: 0.3865 - val_factorized_top_k/top_15_categorical_accuracy: 0.4539 - val_factorized_top_k/top_25_categorical_accuracy: 0.5897 - val_loss: 2350.1934 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2350.1934 - lr: 0.0100
Epoch 9/200
62/62 [==============================] - 3s 44ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8577 - factorized_top_k/top_5_categorical_accuracy: 0.9131 - factorized_top_k/top_10_cat

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8787 - factorized_top_k/top_5_categorical_accuracy: 0.9379 - factorized_top_k/top_10_categorical_accuracy: 0.9669 - factorized_top_k/top_15_categorical_accuracy: 0.9763 - factorized_top_k/top_25_categorical_accuracy: 0.9856 - loss: 1042.6156 - regularization_loss: 0.0000e+00 - total_loss: 1042.6156 - val_factorized_top_k/top_3_categorical_accuracy: 0.2161 - val_factorized_top_k/top_5_categorical_accuracy: 0.2824 - val_factorized_top_k/top_10_categorical_accuracy: 0.3802 - val_factorized_top_k/top_15_categorical_accuracy: 0.4585 - val_factorized_top_k/top_25_categorical_accuracy: 0.5877 - val_loss: 2504.9023 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2504.9023 - lr: 0.0060
Epoch 19/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8782 - factorized_top_k/top_5_categorical_accuracy: 0.9382 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 44ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8863 - factorized_top_k/top_5_categorical_accuracy: 0.9441 - factorized_top_k/top_10_categorical_accuracy: 0.9732 - factorized_top_k/top_15_categorical_accuracy: 0.9810 - factorized_top_k/top_25_categorical_accuracy: 0.9886 - loss: 1015.2757 - regularization_loss: 0.0000e+00 - total_loss: 1015.2757 - val_factorized_top_k/top_3_categorical_accuracy: 0.2141 - val_factorized_top_k/top_5_categorical_accuracy: 0.2743 - val_factorized_top_k/top_10_categorical_accuracy: 0.3759 - val_factorized_top_k/top_15_categorical_accuracy: 0.4600 - val_factorized_top_k/top_25_categorical_accuracy: 0.5879 - val_loss: 2587.7371 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2587.7371 - lr: 0.0036
Epoch 29/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8858 - factorized_top_k/top_5_categorical_accuracy: 0.9439 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8908 - factorized_top_k/top_5_categorical_accuracy: 0.9484 - factorized_top_k/top_10_categorical_accuracy: 0.9761 - factorized_top_k/top_15_categorical_accuracy: 0.9828 - factorized_top_k/top_25_categorical_accuracy: 0.9902 - loss: 1003.8301 - regularization_loss: 0.0000e+00 - total_loss: 1003.8301 - val_factorized_top_k/top_3_categorical_accuracy: 0.2057 - val_factorized_top_k/top_5_categorical_accuracy: 0.2697 - val_factorized_top_k/top_10_categorical_accuracy: 0.3776 - val_factorized_top_k/top_15_categorical_accuracy: 0.4694 - val_factorized_top_k/top_25_categorical_accuracy: 0.5977 - val_loss: 2639.6003 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2639.6003 - lr: 0.0022
Epoch 39/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8907 - factorized_top_k/top_5_categorical_accuracy: 0.9492 - factorized_top_k/top_10_ca

...

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9064 - factorized_top_k/top_5_categorical_accuracy: 0.9548 - factorized_top_k/top_10_categorical_accuracy: 0.9793 - factorized_top_k/top_15_categorical_accuracy: 0.9854 - factorized_top_k/top_25_categorical_accuracy: 0.9919 - loss: 986.8420 - regularization_loss: 0.0000e+00 - total_loss: 986.8420 - val_factorized_top_k/top_3_categorical_accuracy: 0.2069 - val_factorized_top_k/top_5_categorical_accuracy: 0.2654 - val_factorized_top_k/top_10_categorical_accuracy: 0.3716 - val_factorized_top_k/top_15_categorical_accuracy: 0.4772 - val_factorized_top_k/top_25_categorical_accuracy: 0.5986 - val_loss: 2714.5718 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2714.5718 - lr: 1.6927e-06
Epoch 159/200
62/62 [==============================] - 3s 46ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9060 - factorized_top_k/top_5_categorical_accuracy: 0.9549 - factorized_top_k/top_10

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9076 - factorized_top_k/top_5_categorical_accuracy: 0.9552 - factorized_top_k/top_10_categorical_accuracy: 0.9792 - factorized_top_k/top_15_categorical_accuracy: 0.9854 - factorized_top_k/top_25_categorical_accuracy: 0.9920 - loss: 985.0475 - regularization_loss: 0.0000e+00 - total_loss: 985.0475 - val_factorized_top_k/top_3_categorical_accuracy: 0.2066 - val_factorized_top_k/top_5_categorical_accuracy: 0.2640 - val_factorized_top_k/top_10_categorical_accuracy: 0.3730 - val_factorized_top_k/top_15_categorical_accuracy: 0.4758 - val_factorized_top_k/top_25_categorical_accuracy: 0.5989 - val_loss: 2714.6831 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2714.6831 - lr: 1.0156e-06
Epoch 169/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9071 - factorized_top_k/top_5_categorical_accuracy: 0.9555 - factorized_top_k/top_10

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9072 - factorized_top_k/top_5_categorical_accuracy: 0.9555 - factorized_top_k/top_10_categorical_accuracy: 0.9792 - factorized_top_k/top_15_categorical_accuracy: 0.9854 - factorized_top_k/top_25_categorical_accuracy: 0.9919 - loss: 983.5351 - regularization_loss: 0.0000e+00 - total_loss: 983.5351 - val_factorized_top_k/top_3_categorical_accuracy: 0.2089 - val_factorized_top_k/top_5_categorical_accuracy: 0.2640 - val_factorized_top_k/top_10_categorical_accuracy: 0.3730 - val_factorized_top_k/top_15_categorical_accuracy: 0.4766 - val_factorized_top_k/top_25_categorical_accuracy: 0.6003 - val_loss: 2714.7607 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2714.7607 - lr: 1.0000e-06
Epoch 179/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9074 - factorized_top_k/top_5_categorical_accuracy: 0.9553 - factorized_top_k/top_10

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9075 - factorized_top_k/top_5_categorical_accuracy: 0.9553 - factorized_top_k/top_10_categorical_accuracy: 0.9794 - factorized_top_k/top_15_categorical_accuracy: 0.9855 - factorized_top_k/top_25_categorical_accuracy: 0.9919 - loss: 986.4791 - regularization_loss: 0.0000e+00 - total_loss: 986.4791 - val_factorized_top_k/top_3_categorical_accuracy: 0.2052 - val_factorized_top_k/top_5_categorical_accuracy: 0.2634 - val_factorized_top_k/top_10_categorical_accuracy: 0.3713 - val_factorized_top_k/top_15_categorical_accuracy: 0.4749 - val_factorized_top_k/top_25_categorical_accuracy: 0.6009 - val_loss: 2714.8389 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2714.8389 - lr: 1.0000e-06
Epoch 189/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9082 - factorized_top_k/top_5_categorical_accuracy: 0.9552 - factorized_top_k/top_10

62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9075 - factorized_top_k/top_5_categorical_accuracy: 0.9556 - factorized_top_k/top_10_categorical_accuracy: 0.9792 - factorized_top_k/top_15_categorical_accuracy: 0.9855 - factorized_top_k/top_25_categorical_accuracy: 0.9919 - loss: 983.6920 - regularization_loss: 0.0000e+00 - total_loss: 983.6920 - val_factorized_top_k/top_3_categorical_accuracy: 0.2072 - val_factorized_top_k/top_5_categorical_accuracy: 0.2603 - val_factorized_top_k/top_10_categorical_accuracy: 0.3713 - val_factorized_top_k/top_15_categorical_accuracy: 0.4755 - val_factorized_top_k/top_25_categorical_accuracy: 0.5980 - val_loss: 2714.9231 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2714.9231 - lr: 1.0000e-06
Epoch 199/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.9075 - factorized_top_k/top_5_categorical_accuracy: 0.9549 - factorized_top_k/top_10

In [25]:
model.load_weights("new_amazon_check_points/best_check_point_25k")

In [26]:
result_summary = model.evaluate(cached_test)

7/7 [==============================] - 0s 39ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2393 - factorized_top_k/top_5_categorical_accuracy: 0.3182 - factorized_top_k/top_10_categorical_accuracy: 0.4511 - factorized_top_k/top_15_categorical_accuracy: 0.5377 - factorized_top_k/top_25_categorical_accuracy: 0.6410 - loss: 2447.4448 - regularization_loss: 0.0000e+00 - total_loss: 2447.4448


In [27]:
{val:result_summary[idx] for idx, val in enumerate([3, 5, 10,15, 25]) }

{3: 0.2393113374710083,
 5: 0.31822094321250916,
 10: 0.45107603073120117,
 15: 0.5377331376075745,
 25: 0.6410329937934875}

In [28]:
data.history

{'factorized_top_k/top_3_categorical_accuracy': [0.040933433920145035,
  0.17909972369670868,
  0.5759053826332092,
  0.7556427121162415,
  0.8088816404342651,
  0.8333970904350281,
  0.8439173698425293,
  0.8494006395339966,
  0.8576574921607971,
  0.8594108819961548,
  0.8621525168418884,
  0.8648622632026672,
  0.8753825426101685,
  0.8746174573898315,
  0.8757332563400269,
  0.8769127726554871,
  0.8788893222808838,
  0.8787299394607544,
  0.8781560659408569,
  0.8788255453109741,
  0.8786342740058899,
  0.8851058483123779,
  0.8854883909225464,
  0.884850800037384,
  0.886540412902832,
  0.8852971196174622,
  0.8845957517623901,
  0.8862853646278381,
  0.8857753276824951,
  0.8855202794075012,
  0.891099214553833,
  0.8905572295188904,
  0.8904935121536255,
  0.8914817571640015,
  0.8913223743438721,
  0.8895370960235596,
  0.890876054763794,
  0.8908122777938843,
  0.8907485604286194,
  0.8947653770446777,
  0.8951479196548462,
  0.8948609828948975,
  0.8946059942245483,
  0.8953

In [ ]:
keys = iter([key for key in data.history.keys() if key.startswith("factorized_top")])

In [ ]:
x = [i for i in range(1,201,1)]
for key in keys:
    plt.plot(x, data.history[key], "-b", label="Train")
    plt.plot(x, data.history["val_"+key], "-r", label="Test")
    plt.legend(loc="upper left")
    plt.title(key.split("/")[1])
    break

    

In [ ]:
plt.show()